In [11]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

T = 2  # Setze T auf den gewünschten Wert

geo = Box(Pnt(0,0,0), Pnt(1,1,T))  # Box von (0,0,0) bis (1,1,T) # Gitter mit max. Elementgröße 0.2
geo.faces.Max(Z).name = "top"      # Oberseite (z = T)
geo.faces.Min(Z).name = "bottom"   # Unterseite (z = 0)
geo.faces.Min(X).name = "left"     # Seite (x = 0)
geo.faces.Max(X).name = "right"    # Seite (x = 1)
geo.faces.Min(Y).name = "front"    # Seite (y = 0)
geo.faces.Max(Y).name = "back"     # Seite (y = 1)
geo = OCCGeometry(geo) 
mesh = Mesh(geo.GenerateMesh(maxh=0.2)) 
Draw(mesh)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [14]:
order = 6
fes = H1(mesh, order=order, dirichlet="bottom|right|left|front|back")
u,v = fes.TnT()
n = specialcf.normal(3)
w = 0

ux = CoefficientFunction((grad(u)[0],grad(u)[1],0))
B = CoefficientFunction((w,w,1))

a = BilinearForm(fes, symmetric=False)
a += B*grad(u)*v*dx + ux* grad(v)*dx
a.Assemble()

f = LinearForm(fes)
f += 0*v*dx
f.Assemble()

gfu = GridFunction(fes)
gfu.Set(sin(pi*y)*sin(pi*x),BND)  # initial condition
Draw(gfu)

res = f.vec.CreateVector()
res.data = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * res
Draw(gfu)


l2error = sqrt(Integrate((gfu-(sin(pi*x)*sin(pi*y)*exp(-(2*pi**2*z))))**2, mesh))
print(l2error)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

8.12451124764526e-05
